In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/FasterRCNNFiles')

!pip install torchmetrics

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 41.9 MB/s eta 0:00:00


In [ ]:
print(torch.__version__)
print(torchvision.__version__)

2.4.1+cu121
0.19.1+cu121


In [ ]:
import csv

# Define a CSV file to store the loss values
csv_file = '/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNData.csv'

# Create or open the CSV file and write the header (only once)
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Iteration', 'Train loss_classifier', 'Train loss_box_reg', 'Train loss_objectness', 'Train loss_rpn_box_reg', 'Train total_loss',
                     'Val loss_classifier', 'Val loss_box_reg', 'Val loss_objectness', 'Val loss_rpn_box_reg', 'Val total_loss',
                     'Training mAP', 'Training mAP50', 'Training mAP75', 'Training AR1', 'Training AR10', 'Training AR100',
                     'Training map_small', 'Training map_medium', 'Training map_large', 'Training mar_small', 'Training mar_medium', 'Training mar_large',
                     'Validation mAP', 'Validation mAP50', 'Validation mAP75', 'Validation AR1', 'Validation AR10', 'Validation AR100',
                     'Validation map_small', 'Validation map_medium', 'Validation map_large', 'Validation mar_small', 'Validation mar_medium', 'Validation mar_large',
                     'Learning Rate'])

In [ ]:
#verfying the dataset creation and annotations are correct

import torchvision.transforms.functional as F
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.utils import draw_bounding_boxes
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)


#id2label = {1: "nest"}
dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)
img, target = dataset[1]
print(target)
#labels = [for id in target["labels"].tolist()]

#print(labels)

img = draw_bounding_boxes(
    img, target["boxes"], colors="Turquoise", width=2
)
img = F.to_pil_image(img.detach())
img

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


ModuleNotFoundError: No module named 'config'

In [ ]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

class myOwnDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # List: get annotation id from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # path for input image
        path = coco.loadImgs(img_id)[0]["file_name"]
        # open the input image
        img = Image.open(os.path.join(self.root, path))
        # number of objects in the image
        num_objs = len(coco_annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]["bbox"][0]
            ymin = coco_annotation[i]["bbox"][1]
            xmax = xmin + coco_annotation[i]["bbox"][2]
            ymax = ymin + coco_annotation[i]["bbox"][3]
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]["area"])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)


# In my case, just added ToTensor
def get_transform():
    custom_transforms = [torchvision.transforms.ToTensor()]
    #custom_transforms.append(c)
    return torchvision.transforms.Compose(custom_transforms)


# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    #model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1,min_size=640,max_size=640)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
import torch
import torchvision
#import config
#from utils import (
    #get_model_instance_segmentation,
    #ollate_fn,
    #get_transform,
    #myOwnDataset,
#)
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.optim.lr_scheduler import StepLR


print("Torch version:", torch.__version__)

num_epochs = 50

train_data_dir = '/content/drive/MyDrive/niaochao COCO/train'
train_coco = '/content/drive/MyDrive/niaochao COCO/train/_annotations.coco.json'

validation_data_dir = '/content/drive/MyDrive/niaochao COCO/valid'
validation_coco = '/content/drive/MyDrive/niaochao COCO/valid/_annotations.coco.json'

# Training dataset
my_dataset = myOwnDataset(
    root=train_data_dir, annotation=train_coco, transforms=get_transform()
)

# own DataLoader for training dataset
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2,#work around for windows way of handling number of workers
    collate_fn=collate_fn,
)

# Validation dataset
validation_dataset = myOwnDataset(
    root=validation_data_dir, annotation=validation_coco, transforms=get_transform()
)

# own DataLoader for validation dataset
validation_data_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=2,#work around for windows way of handling number of workers
    collate_fn=collate_fn,
)

# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    #print(annotations)

# DataLoader is iterable over Dataset
for vimgs, vannotations in validation_data_loader:
    vimgs = list(img.to(device) for img in vimgs)
    vannotations = [{k: v.to(device) for k, v in t.items()} for t in vannotations]

model = get_model_instance_segmentation(2)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.002,weight_decay=0.0001, momentum=0.9, nesterov=True)
scheduler = StepLR(optimizer=optimizer, step_size=10, gamma=0.5,verbose=True)

len_dataloader = len(data_loader)
len_vdataloader = len(validation_data_loader)

metric_map = MeanAveragePrecision()
metric_mapVal = MeanAveragePrecision()
accuracy = 0.0


print("Model Structure:")
print(model)
# Training
for epoch in range(num_epochs):
    print(f"Epoch: {epoch}/{num_epochs}")
    model.train()
    i = 0
    metric_map.reset()
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)#returns multiple different types of losses from the faster RCNN model
        losses = sum(loss for loss in loss_dict.values())

        # Extract individual loss values
        loss_classifier = loss_dict['loss_classifier'].item()
        loss_box_reg = loss_dict['loss_box_reg'].item()
        loss_objectness = loss_dict['loss_objectness'].item()
        loss_rpn_box_reg = loss_dict['loss_rpn_box_reg'].item()

        # Display each loss from the loss_dict
        #for loss_name, loss_value in loss_dict.items():
          #print(f"{loss_name}: {loss_value.item()}")

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    lr = optimizer.param_groups[0]['lr']
    model.eval()
    with torch.no_grad():
      predictions = model(imgs)
      metric_map.update(predictions, annotations)

    map_metric = metric_map.compute()
    print("Training Metrics:")
    print(map_metric)

    print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")
    #validation after each epoch

    #mAP metrics
    #metric_map.reset()

##########################################################################################################################

    print("Validation on Validation Set:\n")
    model.eval()
    metric_mapVal.reset()
    with torch.no_grad():
      for vimgs, vannotations in validation_data_loader:
        vimgs = list(img.to(device) for img in vimgs)
        vannotations = [{k: v.to(device) for k, v in t.items()} for t in vannotations]

        model.train()
        vloss_dict = model(vimgs, vannotations)
        vlosses = sum(loss for loss in loss_dict.values())

        # Extract individual loss values
        vloss_classifier = vloss_dict['loss_classifier'].item()
        vloss_box_reg = vloss_dict['loss_box_reg'].item()
        vloss_objectness = vloss_dict['loss_objectness'].item()
        vloss_rpn_box_reg = vloss_dict['loss_rpn_box_reg'].item()
        model.eval()

        predictionsv = model(vimgs)
        metric_mapVal.update(predictionsv, vannotations)

      map_metricVal = metric_mapVal.compute()
      print(map_metricVal)

      if(map_metricVal['map'].item()>accuracy):
        torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNBestWeights.pth')
        torch.save(model,'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNModelBestWeights.pt')
        accuracy = map_metricVal['map'].item()

      if(epoch ==  9):
        torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch10.pth')
        torch.save(model,'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch10.pt')

      if(epoch ==  19):
        torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch20.pth')
        torch.save(model,'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch20.pt')

      if(epoch ==  29):
        torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch30.pth')
        torch.save(model,'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch30.pt')

      if(epoch ==  39):
        torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch40.pth')
        torch.save(model,'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch40.pt')

      if(epoch ==  49):
        torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch50.pth')
        torch.save(model,'/content/drive/MyDrive/FasterRCNNFiles/FasterRCNNEpoch50.pt')


      # Save the losses to a CSV file
      with open(csv_file, mode='a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([epoch, i, loss_classifier, loss_box_reg, loss_objectness, loss_rpn_box_reg, losses, vloss_classifier, vloss_box_reg,vloss_objectness,vloss_rpn_box_reg, vlosses,
                         map_metric['map'].item(), map_metric['map_50'].item(), map_metric['map_75'].item(), map_metric['mar_1'].item(), map_metric['mar_10'].item(), map_metric['mar_100'].item(),
                         map_metric['map_small'].item(), map_metric['map_medium'].item(), map_metric['map_large'].item(), map_metric['mar_small'].item(), map_metric['mar_medium'].item(), map_metric['mar_large'].item(),
                         map_metricVal['map'].item(), map_metricVal['map_50'].item(), map_metricVal['map_75'].item(), map_metricVal['mar_1'].item(), map_metricVal['mar_10'].item(), map_metricVal['mar_100'].item(),
                         map_metricVal['map_small'].item(), map_metricVal['map_medium'].item(), map_metricVal['map_large'].item(), map_metricVal['mar_small'].item(), map_metricVal['mar_medium'].item(), map_metricVal['mar_large'].item(),
                         lr])

      print(f"Iteration: {i}/{len_vdataloader}, Val Loss: {vlosses}")

      scheduler.step()


Torch version: 2.4.1+cu121
loading annotations into memory...
Done (t=4.59s)
creating index...
index created!
loading annotations into memory...
Done (t=1.60s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 224MB/s]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Model Structure:
FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640,), max_size=640, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
       

cuda


In [ ]:
import torch
import torchvision
from torchvision import transforms as T

from PIL import Image#reading images
import cv2
from google.colab.patches import cv2_imshow


ig = Image.open('/content/drive/MyDrive/niaochao COCO/test/100042_jpg.rf.28a618c076bfe7a8e2f6c1f0cb038904.jpg')

In [ ]:

transformqw = T.Compose([T.ToTensor()])
img = transformqw(ig)
img = img.to(device)#fixed the issue

with torch.no_grad():
  pred = model([img])

pred

[{'boxes': tensor([[ 572.9694, 1382.7177, 1568.3412, 2079.7786],
          [ 438.5985, 1013.0834, 1743.2255, 2308.6733]], device='cuda:0'),
  'labels': tensor([1, 1], device='cuda:0'),
  'scores': tensor([0.9952, 0.1038], device='cuda:0')}]

In [ ]:
bboxes, labels, scores = pred[0]["boxes"], pred[0]["labels"], pred[0]["scores"]
num = torch.argwhere(scores > 0.7).shape[0]#get where scores greater than 90%
num = 1
bboxes = bboxes.cpu()
igg=cv2.imread('/content/drive/MyDrive/niaochao COCO/test/100042_jpg.rf.28a618c076bfe7a8e2f6c1f0cb038904.jpg')
font=cv2.FONT_HERSHEY_SIMPLEX
for i in range(num):
  x1, y1, x2, y2 = bboxes[i].numpy().astype("int")
  #print(x1, y1, x2, y2)
  #class_name = "nest"#mapping to class names
  igg = cv2.rectangle(igg,(x1,y1),(x2,y2),(0,255,0),1)
  #igg = cv2.putText(igg, class_name, (x1,y1-10),font,0.5,(255,0,0),1,cv2.LINE_AA)

In [ ]:
cv2_imshow(igg)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/FasterRCNNEpoch10ModelPretrainedBatchSize4.pth')

In [ ]:
torch.save(model,'/content/drive/MyDrive/FasterRCNNEpoch10ModelPretrainedBatchSize4.pt')

In [ ]:
#to see model structure
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640,), max_size=640, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(i